# Problems

# Capstone Project
## Structure : 
### 1) Import Libraries 
### 2) 2017 to 2020 
### 3) 2012 to 2016 

# Libraries

In [1]:
# Import Library 

import re
import requests
from re import search

import unicodedata
from bs4 import BeautifulSoup
from bs4 import Tag

from bs4 import NavigableString
from bs4 import SoupStrainer
from pprint import pprint

import numpy as np
from numpy import array
import pandas as pd 

import csv
import pickle

from collections import Counter

from IPython.display import display
from IPython.core import display as ICD
pd.set_option('display.max_columns', 100)


In [2]:
master_filings_dict = {}
        
Accession_Number_URL = {}

AN_Active = {}

name_CIK = { 
    'Tesla' :  ['0001318605']
}

dir_url = r'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK={}&type=10-&dateb=&owner=include&count=100'

doc_url = r'https://www.sec.gov/Archives/edgar/data/{CIKx}/{xx}/{yy}.txt'

In [3]:
# Dict of Companies -> CIK
for name, CIK_list in name_CIK.items():
    
    for CIK_position, CIK in enumerate(CIK_list):
        
        # CIK -> A_N 
        new_dir_url = dir_url.format( CIK )

        new_doc_url = doc_url.format(CIKx = CIK, xx='{xx}', yy='{yy}' )

        response = requests.get( new_dir_url )
        soup = BeautifulSoup( response.content, 'lxml' )
        text = soup.get_text( strip=True )

        cleaned_text = re.findall( 'Acc-no: \d+-\d+-\d+' , text )

        accession_number = [ n.replace('Acc-no: ', '') for n in cleaned_text ]
        accessionnumber = [ num.replace('-', '') for num in accession_number ]
        
        accession_numbers = zip(accessionnumber, accession_number)

        for a,b in accession_numbers: 
            
            file_url_list = new_doc_url.format(xx = a, yy = b)

            Accession_Number_URL.update({ b : file_url_list})
            
print('Accession_Number_URL - ', '\n', Accession_Number_URL)

Accession_Number_URL -  
 {'0001564590-20-019931': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459020019931/0001564590-20-019931.txt', '0001564590-20-018984': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459020018984/0001564590-20-018984.txt', '0001564590-20-004475': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/0001564590-20-004475.txt', '0001564590-19-038256': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019038256/0001564590-19-038256.txt', '0001564590-19-026445': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019026445/0001564590-19-026445.txt', '0001564590-19-013462': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019013462/0001564590-19-013462.txt', '0001564590-19-003165': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019003165/0001564590-19-003165.txt', '0001564590-18-026353': 'https://www.sec.gov/Archives/edgar/data/0001318605/000156459018026353/0001564590-18-026353.txt

## Cleaning Columns Function


In [4]:
month_dict = {'December' : 'Q4',
              'November' : 'Q4',
             'October' : 'Q4',
             'September' : 'Q3',
             'August': 'Q3',
             'July' : 'Q3',
             'June' : 'Q2',
             'May' : 'Q2',
             'April' : 'Q2',
             'March' : 'Q1',
             'Feburary' : 'Q1',
             'January' : 'Q1'}

In [5]:
col_name_variants = {
    'Inventor' : 'Inventory',
    'Total liabilities and' : 'Total liabilities and equity',
    'Common stock.*' : 'Common Stock',
    'Preferred stock.*': 'Preferred Stock',
    'Accounts receivable.*' : 'Accounts Receivable',
    'Accrued liabilities.*' : 'Accrued Liabilities',
    'Deferred revenue.*' : 'Deferred Revenue',
    'Redeemable noncontrolling interests in subsidiaries.*' : 'Noncontrolling interests in subsidiaries',
    'Short term investments.*' : 'Short term investments', 
    'Property|equipment.' : 'Property and Equipment',
    'Retained earnings.*' : 'Retained Earnings',
    'stockholders.*' : 'Shareholder Equity',
    '^Cash.*' : 'Cash',
    'Convertible senior notes.*' : 'Convertible Senior Notes'
}

In [6]:
def clean_col_name( name ):
    
    for detect, replacement in col_name_variants.items():

        if re.findall(detect, name, flags=re.I):

            n_column = name.replace(name, replacement) 

            return n_column 
            
    return name

## Cleaning Tables Function mmmmmmmmmmmmmmmmmmmmmmmmmmm

In [7]:
# Throw column list-like object into cleaning function. 
def cleaning_column(r_column, m):
     
    column_list = []
    column_name = []
    
    for element_post, element in enumerate(r_column):
        
        # Handling the first "row" in the table i.e. column name.
        if element_post == 0:
#             print(element)
            ele_norm = unicodedata.normalize('NFKD', element)
            
            ele_norm = ele_norm.replace('net', '')
            
            ele_norm = ele_norm.replace( '-', ' ')
            
            s_ele_norm = re.sub(r"^\s+|\s+$", "", ele_norm)
            
            name = re.sub(r'(?=,|:|\.|\(){1}.*', repl='', string= s_ele_norm, flags=re.I)
            
            # clean_col_name function Choose old_if or new_dict
            clean_name = clean_col_name( name )
            ###
            column_name.append(clean_name)
            ###
            column_list.append(clean_name)
        
        # Handling the all the remaining "rows" in the table i.e. data/numbers.
        elif element not in ['\n$', '\n\xa0', '\n']:
                        
            n_element = element.replace( ',', '' )

            pattern = re.compile(r'[\d]+,[\d]+|[\d]+')

            res = pattern.findall(n_element)

            res = [int(ele) * m for ele in res]

            column_list.extend(res)

    return column_list
    

In [8]:
# New and imporved.
def Quarter( column ):
    
#     print('column used for quarter', column)
    
    multi_list = [element.split() for element in column if element.split() != []]
#     print('multi_list', multi_list)
    new_column = [item for sublist in multi_list for item in sublist]
#     print('new_column used for year', new_column)
    
    Q = []
    Y = []
    
    for letter in column:
#         print('letter', letter)
        try:
            month = re.findall(r'\w{}+'.format(list(month_dict.keys())), letter.replace(' ',  '') )[0]
            
            Q_num = month_dict[month] 
    #         print('Q_num' , Q_num)
    
            if Q_num != []:
                Q.append(Q_num)
                
        except:
            continue
    #         print('~'*20)
        
#     print('Q' , Q) 
#     Q_final = Q[0] 
#     print('Q_final' , Q_final)
    
    Q_final =['Quarter'] + Q[:1]
    
    
    for letter in new_column:   
        
        #print( letter.split(',') )
        
        Year = [re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) for p_s_ele_norm in letter.split(',') if re.compile(r'^[0-9]{4}$').findall(p_s_ele_norm) != []]
        Y.append(Year)
#     print(f'Y - {Y}')
    Y = [item for sublist in Y for item in sublist][0]
    
    Y.insert(0, 'Year')
    #Y.insert(1, 0)
    #Y.insert(1, 0)
    return Q_final, Y
    print( '\n', 'Q_final', Q_final, 'Y', Y)

In [9]:
A_N = []
for AN, url in Accession_Number_URL.items():
    
    if AN == '0001193125-12-170665':
        break
        
    else:
        A_N.append(AN)
        print(url)
        
print(A_N, '\n')
print(len(A_N))

https://www.sec.gov/Archives/edgar/data/0001318605/000156459020019931/0001564590-20-019931.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000156459020018984/0001564590-20-018984.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/0001564590-20-004475.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000156459019038256/0001564590-19-038256.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000156459019026445/0001564590-19-026445.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000156459019013462/0001564590-19-013462.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000156459019003165/0001564590-19-003165.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000156459018026353/0001564590-18-026353.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000156459018019254/0001564590-18-019254.txt
https://www.sec.gov/Archives/edgar/data/0001318605/000156459018011086/0001564590-18-011086.txt
https://www.sec.gov/Archives/edgar/data/0001318605

## 2017 - 2020

### Creating the .htm urls to get key. post 2016 keys are #BALANCE_SHEETS 

# Testing.
AN_Active = {}
# Get filename, method 3, use htm.# anchor to jump to Balnce sheet.
for AN, url in Accession_Number_URL.items():
    print('~'*75)
    print('url - ', url)
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    
    # if statement takes in Acc-Num format: xx-xxx-xx
    if AN == '0001193125-12-170665':
        break
        
    if int( AN.split( sep='-' )[1] ) > 12:
        
        r = requests.get(url)
        
        try: 
            soup = BeautifulSoup(r.content, "html")
            
        except:
            print('Look at: ')
            print( '{} - {}'.format(AN, url))
            
        filing_document = soup.find('document')
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        print('document_id - ', document_id)
        
        if document_id == '10-Q' or document_id == '10-K':
            
            # 1.0
            AccessionNumber = AN.replace('-', '')

            # 2.1 part of table website.
            filename = str(soup.find('filename').contents[0]).replace('\n', '')
            print('filename - ', filename, '\n')

            # 2.2 Anchor page_key.
            hr_tag = soup.findAll('a', href=True)
            page_key = str( [table.get('href') for table in hr_tag if 'Balance Sheets' in table.get_text()][0])
            print('page_key - ', page_key, '\n')


AN_Active = {}

# Get filename, method 3, use htm.# anchor to jump to Balnce sheet.
for AN, url in Accession_Number_URL.items():
    
    print('~'*75)
    print('url - ', url)
    
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    
    # if statement takes in Acc-Num format: xx-xxx-xx
        
#     if int( AN.split( sep='-' )[1] ) > 12:

    r = requests.get(url)

    try: 
        soup = BeautifulSoup(r.content, "html")

    except:
        print('Look at: ')
        print( '{} - {}'.format(AN, url))

    filing_document = soup.find('document')
    document_id = filing_document.type.find(text=True, recursive=False).strip()
    print('document_id - ', document_id)

    if document_id == '10-Q' or document_id == '10-K':

        # 1.0
        AccessionNumber = AN.replace('-', '')

        # 2.1 part of table website.
        filename = str(soup.find('filename').contents[0]).replace('\n', '')
        print('filename - ', filename, '\n')

        # 2.2 Anchor page_key.
        hr_tag = soup.findAll('a', href=True)
        page_key = str( [table.get('href') for table in hr_tag if 'Balance Sheets' in table.get_text()][0])
        print('page_key - ', page_key, '\n')

        KEY = filename + page_key

        active_url = new_doc_url.format( xx = AccessionNumber, yy = KEY ).replace('.txt', '')

#         print('active_url - ', active_url)

        # Key for 2016 onwards is the same, will overwrite the keys() in AN_Active. 
        AN_Active.update({ AN : [ page_key , url ] } )

    else:
        continue


In [11]:
AN_Active = {'0001564590-20-019931': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459020019931/0001564590-20-019931.txt'],
 '0001564590-20-004475': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/0001564590-20-004475.txt'],
 '0001564590-19-038256': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019038256/0001564590-19-038256.txt'],
 '0001564590-19-026445': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019026445/0001564590-19-026445.txt'],
 '0001564590-19-013462': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019013462/0001564590-19-013462.txt'],
 '0001564590-19-003165': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019003165/0001564590-19-003165.txt'],
 '0001564590-18-026353': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459018026353/0001564590-18-026353.txt'],
 '0001564590-18-019254': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459018019254/0001564590-18-019254.txt'],
 '0001564590-18-011086': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459018011086/0001564590-18-011086.txt'],
 '0001564590-18-002956': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459018002956/0001564590-18-002956.txt'],
 '0001564590-17-021343': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459017021343/0001564590-17-021343.txt'],
 '0001564590-17-015705': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459017015705/0001564590-17-015705.txt'],
 '0001564590-17-009968': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459017009968/0001564590-17-009968.txt'],
 '0001564590-17-003118': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459017003118/0001564590-17-003118.txt'],
 '0001564590-16-026820': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459016026820/0001564590-16-026820.txt'],
 '0001564590-16-023024': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459016023024/0001564590-16-023024.txt'],
 '0001564590-16-018886': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459016018886/0001564590-16-018886.txt'],
 '0001564590-16-013195': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459016013195/0001564590-16-013195.txt'],
 '0001564590-15-009741': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459015009741/0001564590-15-009741.txt'],
 '0001564590-15-006666': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459015006666/0001564590-15-006666.txt'],
 '0001564590-15-003789': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459015003789/0001564590-15-003789.txt'],
 '0001564590-15-001031': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459015001031/0001564590-15-001031.txt'],
 '0001193125-14-403635': ['#tx812482_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312514403635/0001193125-14-403635.txt'],
 '0001193125-14-303175': ['#tx766922_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312514303175/0001193125-14-303175.txt'],
 '0001193125-14-192606': ['#tx715897_2',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312514192606/0001193125-14-192606.txt'],
 '0001193125-14-069681': ['#tx668062_26',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312514069681/0001193125-14-069681.txt'],
 '0001193125-13-435480': ['#tx588506_2',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312513435480/0001193125-13-435480.txt'],
 '0001193125-13-327916': ['#tx549636_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312513327916/0001193125-13-327916.txt'],
 '0001193125-13-212354': ['#tx511008_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312513212354/0001193125-13-212354.txt'],
 '0001193125-13-096241': ['#tx452995_102',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312513096241/0001193125-13-096241.txt'],
 '0001193125-12-457610': ['#tx410318_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312512457610/0001193125-12-457610.txt'],
 '0001193125-12-332138': ['#tx364775_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312512332138/0001193125-12-332138.txt'],
 '0001193125-12-225825': ['#tx325967_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312512225825/0001193125-12-225825.txt'],
 '0001193125-12-081990': ['#tx279413_101',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312512081990/0001193125-12-081990.txt'],
 '0001193125-11-308489': ['#tx226201_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312511308489/0001193125-11-308489.txt'],
 '0001193125-11-221497': ['#tx198925_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312511221497/0001193125-11-221497.txt'],
 '0001193125-11-139677': ['#toc186744_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312511139677/0001193125-11-139677.txt'],
 '0001193125-11-054847': ['#tx151489_23',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312511054847/0001193125-11-054847.txt'],
 '0001193125-10-259068': ['#toc108111_3',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000119312510259068/0001193125-10-259068.txt']}

In [ ]:
print( [txt for acc_num,  txt in Accession_Number_URL.items() if 21 > int(acc_num.split(sep='-')[1]) > 16] )

In [ ]:
# Microsoft - AN of new anchor tag - 'id. 
print("Anchor tag change from 'name' to 'id' for the following Accession Number: \n")
list_AN_Active = list(AN_Active.keys())
list_AN_Active.reverse() 
name_to_id = list_AN_Active[-3:]
print(f'{name_to_id} \n')

print("Assuming all new filings are 'id' tagged.")

In [ ]:
AN_active_test = { '0001564590-19-003165': ['#Consolidated_Balance_Sheets',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019003165/0001564590-19-003165.txt']}

# 10-K:  Following works for '20-004475', but not for '19-003165'
## '19-003165' tag is 'name'

In [ ]:
only_attr_with_name = SoupStrainer( attrs={"name":key} )
only_attr_with_id = SoupStrainer(attrs={"id": key} )


In [ ]:
Checklist ={}

# Just trying things for '19-003165'
# 10-K - '0001564590-20-004475', '0001564590-19-003165'
AN_active_test = {'0001564590-19-013462': ['#CONSOLIDATED_BALANCE_SHEETS',
  'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019013462/0001564590-19-013462.txt']}

# 2016 onwards. Use the above Accession_Number_URL_16_20, 2016+ need to use .txt format. 
for AN, key_url in AN_Active.items():
    year = int( AN.split( sep='-' )[1] )
#     m= 10**3
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    print('~'*75)

    # if statement takes in Acc-Num: xx-xxx-xx
    if AN == '0001193125-12-170665':
        break

    if year < 21:
        
        print(f'Accession Number - {AN}\n')
        key = key_url[0].replace('#', '')
        print(f'key - {key}\n')
        act_url = key_url[1]
        print(f'Active URL - {key_url[1]}\n')
        
        r = requests.get(act_url)
        
        # Not choosing r.encoding to easier modify 
#         r.encoding = 'utf-8'
        
        soup = BeautifulSoup(r.content, "lxml")

        filing_document = soup.find('document')
        document_id = filing_document.type.find(text=True, recursive=False).strip()
#         print(f'document_id - {document_id}\n' )
        
        table = None
        multiplier = None
        
        page_key = soup.find(attrs={"name": key} )
        
        if (page_key != None):
            print(f'page_key name - {page_key}\n' )
    
        else:
            page_key = soup.find(attrs={"id": key} )
            print(f'page_key id - {page_key}\n' )
            
        getting_nextP(page_key, n=0)
        getting_m(page_key, n=0)

        target = table.findAll('tr')
#         print(f'target - {target[0]}\n')

        row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']
#         print(f'row_list - {row_list[0]}\n')

        col_list = []

        for row in row_list:
            col = row.find_all('td')
#             print(f'col - {col}')
            column = []
            for tr in col:
#                 print(f'tr - {tr}')
                ele = tr.text
#                 print(f'ele - {ele}')
#                 try:
#                     ele= ele.encode('cp1252').decode("utf-8", "ignore")
#                 except:
#                     ele.encode('latin1').decode("utf-8", "ignore")

                column.append(ele)
#             print(f'column - {column}')
            col_list.append(column)

#         col_list.pop(0)

        new_col_list = []
        
        
        # Tesla Date is split to Q[0] and Y[1].
        col_list[0] = col_list[0] + col_list[1]
        col_list.pop(1)
        
        for col_num, column in enumerate(col_list): 

            if col_num == 0:
#                 print('raw column - ', column)
                Q, Y = Quarter(column)
                print(Q, Y)
                new_col_list.append(Q)
                new_col_list.append(Y)

            else:
                column.pop(1)
                column.pop(1)
#                 print(f'column - {column}')
                new_col_list.append(cleaning_column( column, multiplier ))

        df = pd.DataFrame(new_col_list)
        df = df.T.reset_index(drop=True)
        df.columns = df.iloc[0]
        df = df.drop([row for row in range(df.shape[0]) if row != 1], axis=0).reset_index(drop=True)
        ICD.display(df)

        with open('{}-{}.csv'.format(AN, document_id), "wb") as fp:   # Pickling
            pickle.dump(df, fp)

        # display a status to the user.
        print('All the pages from {} document {} have been tableized and saved.'.format(AN, document_id))    
        print('-'*80) 

if Checklist == {}:
    print('\nEverything went well.\n')

else: 
    print('\n******************These files have a problem.******************\n', Checklist)

In [3]:
print( soup.find(attrs={"name": 'CONSOLIDATED_BALANCE_SHEETS'} ).parent)

<p style="text-align:center;margin-bottom:0pt;margin-top:4pt;text-indent:0%;font-weight:bold;font-family:Times New Roman;font-size:10pt;font-style:normal;text-transform:none;font-variant: normal;"><a name="CONSOLIDATED_BALANCE_SHEETS"></a><a name="CONSOLIDATED_BALANCE_SHEETS"></a>Consolidated Balance Sheets</p>


In [ ]:
from re import search

test_str =  'mMMMillion in units'
if search(r'Million', test_str, flags=re.I):
    print('true')

In [12]:
# Multiplier finder.
def finding_m(current_sibling, nex_sibling):
    
    global multiplier
#     print(f'current_sibling - {current_sibling}')

    
    # Base Case
    if search(r'thousand', current_sibling.text, flags=re.I):
        multiplier = 10**3
    #         print(f'current_sibling - {current_sibling.text}')
        return multiplier
    
    if search(r'Million', current_sibling.text, flags=re.I):
        multiplier = 10**6
#         print(f'current_sibling - {current_sibling.text}')
        return multiplier

    
    else:
        return finding_m(nex_sibling, nex_sibling.next_sibling)


In [13]:
table = None
def getting_m(page, n):
    
#     print('page{}'.format('.parent'*n))
    
    while n<4:
        
        try: #try the current tag level.
                
            return finding_m(page, page.next_sibling)
        
        except AttributeError: # Handling None type object has no 'next_sibling' exception. 

#             print('\n --- Trying the next parent --- \n')
            n += 1
            getting_m(page.parent, n) # try the same function again, this time use the parent-tag.

In [18]:
getting_m(page_key, n=0)

In [19]:
multiplier

1000

In [14]:
# Working next_sibling finder.
def nextS(current_sibling, nex_sibling):
    
    global table
#     print(f'current_sibling - {current_sibling}')
    
    # Base Case
    if (current_sibling.name == 'table' or current_sibling.name =='div'):
        table = current_sibling
#         print(f'current_sibling - {current_sibling.text}')
        return current_sibling
    
    else:
        return nextS(nex_sibling, nex_sibling.next_sibling)


In [15]:
table = None
def getting_nextP(page, n):
    
#     print('page{}'.format('.parent'*n))
    
    while n<4:
        
        try: #try the current tag level.
                
            return nextS(page, page.next_sibling)
        
        except AttributeError: # Handling None type object has no 'next_sibling' exception. 

#             print('\n --- Trying the next parent --- \n')
            n += 1
            getting_nextP(page.parent, n) # try the same function again, this time use the parent-tag.

In [ ]:
getting_nextP(page_key, n=0)

In [ ]:
print(table)

In [ ]:
print(page_key.parent.parent)

# Don't touch this 

In [ ]:
  
# 10-K - '0001564590-20-004475', '0001564590-19-003165'
AN_active_test = {'0001564590-20-004475': ['#Consolidated_Balance_Sheets',
        'https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/0001564590-20-004475.txt']}
                                           
# 2016 onwards. Use the above Accession_Number_URL_16_20, 2016+ need to use .txt format. 
for AN, key_url in AN_active_test.items():
    year = int( AN.split( sep='-' )[1] )
#     m= 10**3
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    print('~'*75)

    # if statement takes in Acc-Num: xx-xxx-xx
    if AN == '0001193125-12-170665':
        break

    if year < 21:
        
        print(f'Accession Number - {AN}\n')
        key = key_url[0]
        print(f'key - {key_url[0]}\n')
        act_url = key_url[1]
        print(f'Active URL - {key_url[1]}\n')
        
        r = requests.get(act_url)
        soup = BeautifulSoup(r.content, "lxml")

        filing_document = soup.find('document')
#         print('filing_document - ', filing_document)
        
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        print(f'document_id - {document_id}\n' )

        page_key = soup.find_all(attrs={"id": "{}".format(key.replace('#', '') ) } )
        print(key)
#         print(f'page_key - {page_key}\n' )
# T_table = [ table.parent.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]

        Multiplier_code = [ table.next_sibling.next_sibling for table in page_key ][0]
        print(f'Multiplier_code - {Multiplier_code}', '////type of Multiplier_code - {}'.format(type(Multiplier_code)))
        m = m_calculator(Multiplier_code)
#         print(f'm - {m}')
#         break
        T_table = [ table.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
#         print(f'T_table - {T_table}', '////type of T_table - {}'.format(type(T_table)))

        target = T_table[0].findAll('tr')
#         print(f'target - {target[0]}\n')
        
        row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']
#         print(f'row_list - {row_list[0]}\n')

        col_list = []

        for row in row_list:
            col = row.find_all('td')
#             print(f'col - {col}')
            column = []
            for tr in col:
                ele = tr.text
#                 print(f'ele - {ele}')
                try:
                    ele= ele.encode('cp1252').decode("utf-8", "ignore")
                except:
                    ele.encode('latin1').decode("utf-8", "ignore")

                column.append(ele)
#             print(f'column - {column}')
            col_list.append(column)

#         col_list.pop(0)

        new_col_list = []
        
        
        # Tesla Date is split to Q[0] and Y[1].
        col_list[0] = col_list[0] + col_list[1]
        col_list.pop(1)
        
        for col_num, column in enumerate(col_list): 

            if col_num == 0:
#                 print('raw column - ', column)
                Q, Y = Quarter(column)
                print(Q, Y)
                new_col_list.append(Q)
                new_col_list.append(Y)

            else:
                column.pop(1)
                column.pop(1)
#                 print(f'column - {column}')
                new_col_list.append(cleaning_column( column, m ))

        df = pd.DataFrame(new_col_list)
        df = df.T.reset_index(drop=True)
        df.columns = df.iloc[0]
        df = df.drop([row for row in range(df.shape[0]) if row != 1], axis=0).reset_index(drop=True)
        ICD.display(df)


In [16]:
def m_calculator( Multiplier_code ):
    
    Multiplier_text = Multiplier_code.get_text(strip=True, separator=',')
    print(f'Multiplier_text - {Multiplier_text}' )

    if bool(re.search(r'million.', Multiplier_text, flags=re.I)) == True:
        m = 10**6
        print(f'm - {m}' )
        return m
    
    elif bool(re.search(r'thousand.', Multiplier_text, flags=re.I)) == True:
        m = 10**3
        print(f'm - {m}' )
        return m
    
    else: 
        print(f'Multiplier_text - {Multiplier_text}' )
        return 0
     

In [17]:
def K_Table_creator( soup, key ):

#Consolidated_Balance_Sheets
    try:
        
        page_key = soup.find_all(attrs={"name": "{}".format(key.replace('#', '') ) } )
#         print(f'"name"  - {page_key}\n' )

        Multiplier_code = [ table.parent.next_sibling.next_sibling for table in page_key ][0]
#         print(f'Multiplier_code - {Multiplier_code}', '////type of Multiplier_code - {}'.format(type(Multiplier_code)))
        m = m_calculator(Multiplier_code)
#         print(f'm - {m}')
        
        T_table = [ table.parent.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
#         print(f'T_table - {T_table}', '////type of T_table - {}'.format(type(T_table)))
        return T_table, m

    except:
        page_key = soup.find_all(attrs={"id": "{}".format(key.replace('#', '') ) } )
#         print(f'page_key - {page_key}\n' )

        Multiplier_code = [ table.next_sibling.next_sibling for table in page_key ][0]
#         print(f'Multiplier_code - {Multiplier_code}', '////type of Multiplier_code - {}'.format(type(Multiplier_code)))
        m = m_calculator(Multiplier_code)
#         print(f'm - {m}')

        T_table = [ table.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
#         print(f'T_table - {T_table}', '////type of T_table - {}'.format(type(T_table)))
        return T_table, m

    else: 
        m = 0
        return m

In [18]:
def Q_Table_creator( soup, key ):

#Consolidated_Balance_Sheets
    try:
        print('<html> attr   -    (id - Consolidated_Balance_Sheets) \n')
        page_key = soup.find_all(attrs={"id": key } )
#         print(f'page_key - {page_key}\n' )

        Multiplier_code = [ table.next_sibling.next_sibling for table in page_key ][0]
#         print(f'Multiplier_code - {Multiplier_code}', 'type of Multiplier_code - {}'.format(type(Multiplier_code)))
        m= m_calculator(Multiplier_code)
    
        T_table = [ table.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling 
                   for table in page_key ]
#         print(f'T_table - {T_table}')
        return T_table, m

# CONSOLIDATED_BALANCE_SHEETS HANDLING 0001564590-15-009741
    except:
        print('<html> attr   -    (name - CONSOLIDATED_BALANCE_SHEETS) \n')
        page_key = soup.find_all(attrs={"name": key } )
        print(f'page_key - {page_key}\n' )

        Multiplier_code = [ table.parent.next_sibling.next_sibling for table in page_key ][0]
#         print(f'Multiplier_code - {Multiplier_code}', 'type of Multiplier_code - {}'.format(type(Multiplier_code)))
        m= m_calculator(Multiplier_code)

        T_table = [ table.parent.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling
                   for table in page_key ]
        print(f'T_table.name - {T_table[0].name}')
#         print(f'T_table - {T_table}')
        return T_table, m
    
# The Following valuable code works up until    0001564590-15-009741
# CONSOLIDATED_BALANCE_SHEETS
#     except:
#         print('<html> attr   -    (name - CONSOLIDATED_BALANCE_SHEETS) \n')
#         page_key = soup.find_all(attrs={"name": key } )
# #         print(f'page_key - {page_key}\n' )

#         Multiplier_code = [ table.parent.next_sibling.next_sibling for table in page_key ][0]
# #         print(f'Multiplier_code - {Multiplier_code}', 'type of Multiplier_code - {}'.format(type(Multiplier_code)))
#         m= m_calculator(Multiplier_code)

#         T_table = [ table.parent.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling
#                    for table in page_key ]
#         print(f'T_table - {T_table}')
#         return T_table, m

    else:

# txxxxxx_x
        if bool(re.search(r'tx.', key, flags=re.I)) == True:
            
            print('<html> attr   -    (name - txxxxxx_x) \n')
            page_key = soup.find(attrs={"name": key })
    #         print(f'page_key - {page_key}\n' )

            Multiplier_code = [ table.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ][0]
            m= m_calculator(Multiplier_code)

            T_table = [ table.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
    #         print(f'T_table - {T_table}')
            return T_table, m

# tocxxxxxx_x
        elif bool(re.search(r'toc.', key, flags=re.I)) == True:
        
            print('<html> attr   -    (name - tocxxxxxx_x) \n')
            page_key = soup.find_all(attrs={"name": key })
            print(key)
#             print(f'page_key - {page_key}\n' )
            Multiplier_code = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ][0]

            m= m_calculator(Multiplier_code)

            T_table = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
    #         print(f'T_table - {T_table}')
            return T_table, m
        
        else: 
            m = 0
            return m

In [21]:
Checklist ={}
# AN_active_test = {'0001564590-19-013462': ['#CONSOLIDATED_BALANCE_SHEETS',
#   'https://www.sec.gov/Archives/edgar/data/0001318605/000156459019013462/0001564590-19-013462.txt']}
# 2016 onwards. Use the above Accession_Number_URL_16_20, 2016+ need to use .txt format. 
for AN, key_url in AN_Active.items():
    year = int( AN.split( sep='-' )[1] )
    m=0
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    print('~'*75)

    # if statement takes in Acc-Num: xx-xxx-xx
#     if AN == '0001193125-12-170665':
#         break
    
    print(f'Accession Number - {AN}\n')
    key = key_url[0].replace('#', '')
    print(f'key - {key}\n')
    act_url = key_url[1]
    print(f'Active URL - {act_url}\n')

    r = requests.get(act_url)
    soup = BeautifulSoup(r.content, "lxml")

    filing_document = soup.find('document')

    document_id = filing_document.type.find(text=True, recursive=False).strip()
    print(f'document_id - {document_id}\n' )

#                                         -------->Table_creater<--------
    if document_id == '10-Q':
        print(key)
        T_table, m = Q_Table_creator(soup, key)

    if document_id == '10-K':
        print(key)
        T_table, m = K_Table_creator(soup, key)
    
    target = T_table[0].findAll('tr')
    row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']

    col_list = []

    for row in row_list:
        col = row.find_all('td')
        column = []
        for tr in col:
            ele = tr.text
            
            try:
                ele= ele.encode('cp1252').decode("utf-8", "ignore")
            except:
                ele.encode('latin1').decode("utf-8", "ignore")
                
            column.append(ele)
        col_list.append(column)

#     col_list.pop(0)

    new_col_list = []
    
    # Tesla Date is split to Q[0] and Y[1].
    try:
        col_list[0] = col_list[0] + col_list[1]
        col_list.pop(1)
        
    except:
        print(type(document_id), type(AN))
        Checklist.update( { document_id : AN })
        print("Can't save file {}, file type - {}".format(AN, document_id))
        continue
    for col_num, column in enumerate(col_list): 

        if col_num == 0:
#             print('raw column - ', column)
            Q, Y = Quarter(column)
#             print(Q, Y)
            new_col_list.append(Q)
            new_col_list.append(Y)

        else:
            column.pop(1)
            column.pop(1)

            new_col_list.append(cleaning_column( column, m ))

    df = pd.DataFrame(new_col_list)
    df = df.T.reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df.drop([row for row in range(df.shape[0]) if row != 1], axis=0).reset_index(drop=True)
    ICD.display(df)
 
#     with open('{}-{}.csv'.format(AN, document_id), "wb") as fp:   # Pickling
#         pickle.dump(df, fp)

    # display a status to the user.
    print('All the pages from {} document {} have been tableized and saved.'.format(AN, document_id))    
    print('-'*80) 

if Checklist == {}:
    print('\nEverything went well.\n')
    
else: 
    print('\n******************These files have a problem.******************\n', Checklist)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001564590-20-019931

key - Consolidated_Balance_Sheets

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000156459020019931/0001564590-20-019931.txt

document_id - 10-Q

Consolidated_Balance_Sheets
<html> attr   -    (id - Consolidated_Balance_Sheets) 

Multiplier_text - (in millions, except per share data)
m - 1000000


,Quarter,Year,Assets,Current assets,Cash,Accounts Receivable,Inventory,Prepaid expenses and other current assets,Total current assets,Operating lease vehicles,Solar energy systems,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,Other non current assets,Total assets,Liabilities,Current liabilities,Accounts payable,Accrued Liabilities,Deferred Revenue,Customer deposits,Current portion of debt and finance leases,Total current liabilities,Debt and finance leases,Deferred Revenue,Other long term liabilities,Total liabilities,Commitments and contingencies,Noncontrolling interests in subsidiaries,Convertible Senior Notes,Equity,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Accumulated deficit,Shareholder Equity,Noncontrolling interests in subsidiaries,Total liabilities and equity
0,Q1,2020,None,None,8080000000,1274000000,4494000000,1045000000,14893000000,2527000000,6106000000,10638000000,1197000000,323000000,193000000,1373000000,37250000000,None,None,3970000000,2825000000,1186000000,788000000,3217000000,11986000000,10666000000,1199000000,2667000000,26518000000,None,632000000,60000000,None,None,None,0,15390000000,113000000,6104000000,9173000000,867000000,37250000000


All the pages from 0001564590-20-019931 document 10-Q have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001564590-20-004475

key - Consolidated_Balance_Sheets

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000156459020004475/0001564590-20-004475.txt

document_id - 10-K

Consolidated_Balance_Sheets
Multiplier_text - (in millions, except per share data)
m - 1000000


,Quarter,Year,Assets,Current assets,Cash,Restricted cash,Accounts Receivable,Inventory,Prepaid expenses and other current assets,Total current assets,Operating lease vehicles,Solar energy systems,Property and Equipment,Operating lease right of use assets,Intangible assets,Goodwill,MyPower customer notes receivable,Restricted cash,Other assets,Total assets,Liabilities,Current liabilities,Accounts payable,Accrued Liabilities,Deferred Revenue,Resale value guarantees,Customer deposits,Current portion of debt and finance leases,Total current liabilities,Debt and finance leases,Deferred Revenue,Resale value guarantees,Other long term liabilities,Total liabilities,Commitments and contingencies,Noncontrolling interests in subsidiaries,Equity,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Accumulated deficit,Shareholder Equity,Noncontrolling interests in subsidiaries,Total liabilities and equity
0,Q4,2019,None,None,6268000000,246000000,1324000000,3552000000,713000000,12103000000,2447000000,6138000000,10396000000,1218000000,339000000,198000000,393000000,269000000,808000000,34309000000,None,None,3771000000,2905000000,1163000000,317000000,726000000,1785000000,10667000000,11634000000,1207000000,36000000,2655000000,26199000000,None,643000000,None,None,None,0,12737000000,36000000,6083000000,6618000000,849000000,34309000000


All the pages from 0001564590-20-004475 document 10-K have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001564590-19-038256

key - CONSOLIDATED_BALANCE_SHEETS

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000156459019038256/0001564590-19-038256.txt

document_id - 10-Q

CONSOLIDATED_BALANCE_SHEETS
<html> attr   -    (id - Consolidated_Balance_Sheets) 

Multiplier_text - (in millions, except for par values)
m - 1000000


...

,Quarter,Year,,Assets,Current assets,Cash,Restricted cash and marketable securities,Accounts Receivable,Inventory,Prepaid expenses and other current assets,Total current assets,Operating lease vehicles,Property and Equipment,Restricted cash,Other assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Accrued Liabilities,Deferred Revenue,Capital lease obligations,Customer deposits,Convertible Senior Notes,Total current liabilities,Capital lease obligations,Deferred Revenue,Convertible Senior Notes,Resale value guarantee,Other long term liabilities,Total liabilities,Commitments and contingencies,Convertible Senior Notes,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Accumulated other comprehensive loss,Accumulated deficit,Shareholder Equity,Total liabilities and equity
0,Q1,2015,None,None,None,1510076000,20693000,200052000,1054840000,135756000,2921417000,912061000,2224191000,13846000,48515000,6120030000,None,None,732331000,353768000,226474000,9622000,249476000,620710000,2192381000,11265000,312850000,1888672000,606221000,228367000,5239756000,None,54277000,None,None,126000,2429677000,15965000,1587841000,825997000,6120030000


All the pages from 0001564590-15-003789 document 10-Q have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001564590-15-001031

key - Consolidated_Balance_Sheets

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000156459015001031/0001564590-15-001031.txt

document_id - 10-K

Consolidated_Balance_Sheets
Multiplier_text - (in thousands, except share and per share data)
m - 1000


,Quarter,Year,Assets,Current assets,Cash,Restricted cash and marketable securities,Accounts Receivable,Inventory,Prepaid expenses and other current assets,Total current assets,Operating lease vehicles,Property and Equipment,Restricted cash,Other assets,Total assets,Shareholder Equity,Current liabilities,Accounts payable,Accrued Liabilities,Deferred Revenue,Capital lease obligations,Customer deposits,Convertible Senior Notes,Total current liabilities,Capital lease obligations,Deferred Revenue,Convertible Senior Notes,Resale value guarantee,Other long term liabilities,Total liabilities,Commitments and contingencies,Convertible Senior Notes,Shareholder Equity,Preferred Stock,Common Stock,Additional paid in capital,Accumulated deficit,Shareholder Equity,Total liabilities and equity
0,Q4,2014,None,None,1905713000,17947000,226604000,953675000,94718000,3198657000,766744000,1829267000,11374000,43209000,5849251000,None,None,777946000,268884000,191651000,9532000,257587000,601566000,2107166000,12267000,292271000,1806518000,487879000,173244000,4879345000,None,58196000,None,None,126000,2345266000,1433682000,911710000,5849251000


All the pages from 0001564590-15-001031 document 10-K have been tableized and saved.
--------------------------------------------------------------------------------
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Accession Number - 0001193125-14-403635

key - tx812482_3

Active URL - https://www.sec.gov/Archives/edgar/data/0001318605/000119312514403635/0001193125-14-403635.txt

document_id - 10-Q

tx812482_3
<html> attr   -    (id - Consolidated_Balance_Sheets) 

<html> attr   -    (name - CONSOLIDATED_BALANCE_SHEETS) 

page_key - [<a name="tx812482_3"></a>]



AttributeError: 'NoneType' object has no attribute 'next_sibling'

In [ ]:
T_table = [ table.parent.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling.next_sibling
                   for table in page_key ]


In [ ]:
print(page_key)

In [ ]:
page_key = soup.find(attrs={"name": "{}".format(key.replace('#', '') ) } )


In [ ]:
print( table.text )

In [ ]:
key_tag = page_key.parent.parent
print(key_tag)

# print outs from previous work

In [ ]:
print('\n', table)

In [ ]:
# Working next_sibling finder.
def nextS(prev, n, x, y):
    
    
    global BalancedSheet
    # Base Case
    if y==1:
        return BalancedSheet
    
    print(f'Parent {n} Child {x}')
    print('#'*30)
    
    x+=1
    
    #Target is accuired. 
    while (isinstance(prev, Tag) and prev.name == 'table' and y ==0):
         
        BalancedSheet = prev
        print(BalancedSheet.get_text())
        print('jackpot!')
        y +=1
        
    # Tags are ignored and add a next_sibling.
    while (isinstance(prev, Tag) and y<1):
        print(' --- Below is a Tag --- \n', Tag )
        nextS(prev.next_sibling, n, x, y)
        
    # NavigableString are ignored and add a next_sibling.
    while (isinstance(prev, NavigableString) and y<1):
        print(' ~~~ NavigableString was here ~~~ ')
        nextS(prev.next_sibling, n, x, y)
        
    else:
        if (y==0):
            print(f'Please print this - Child {x}')
            nextS(prev.next_sibling, n, x, y)
            
        else:
            return BalancedSheet

##### 2017- 2020 working material

In [ ]:
type(column[0])

In [ ]:
url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459019012709/0001564590-19-012709.txt'
r = requests.get(url)
soup = BeautifulSoup(r.content, "html.parser")

page_key = soup.find_all(attrs={"name": "BALANCE_SHEETS"})
    
T_table = [page_key[0].parent.next_sibling.next_sibling.next_sibling.next_sibling ]
target = T_table[0].findAll('tr')
print(target)

In [ ]:
page_key

In [ ]:
print([row for row in range(df.shape[0]) if row != 1])

In [ ]:
[table.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]

In [ ]:
T_table = [ table.next_sibling.next_sibling.children for table in page_key ]
print(T_table)

## End 2017-2020

In [ ]:
# Show Accession_Number_URL
Accession_Number_URL

## 2012 -2016

### Creating the .htm urls to get key.

In [ ]:
AN_Active = {}
# Get filename, method 3, use htm.# anchor to jump to Balnce sheet.
for AN, url in Accession_Number_URL.items():
    
    # Break is needed to limit lower bound, either 
    # pre-2016 = 0001193125-12-170665 (if excludes -12-170665 ) or 
    # post-16 = 0001193125-16-742796 (x > 16, excludes 16)
    
    # if statement takes in Acc-Num: xx-xxx-xx
    if AN == '0001193125-12-170665':
        break
        
    if int( AN.split( sep='-' )[1] ) < 17:
        print('~'*75)
        r = requests.get(url)
        
        try: 
            soup = BeautifulSoup(r.content, "html")
            
        except:
            print('Look at: ')
            print( '{} - {}'.format(AN, url))
            
        # 1.0
        AccessionNumber = AN.replace('-', '')
        
        # 2.1 part of table website.
        filename = str(soup.find('filename').contents[0]).replace('\n', '')
        print('filename - ', filename, '\n')
        
        # 2.2 Anchor page_key.
        hr_tag = soup.findAll('a', href=True)
        page_key = str( [table.get('href') for table in hr_tag if 'Balance Sheets' in table.get_text()][0])
        print('page_key - ', page_key, '\n')
        
        KEY = filename + page_key
        
        active_url = new_doc_url.format( xx = AccessionNumber, yy = KEY ).replace('.txt', '')

        print('active_url - ', active_url)
        
        # Key for 2016 onwards is the same, will overwrite the keys() in AN_Active. 
        AN_Active.update({ page_key : [active_url, AN]})

    else:
        continue
    

##### 2012- 2016 working material

AN_Active = {
 '0001193125-16-742796': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312516742796/0001193125-16-742796.txt'],
 '0001193125-16-662209': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312516662209/0001193125-16-662209.txt'],
 '0001193125-16-550254': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312516550254/0001193125-16-550254.txt'],
 '0001193125-16-441821': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312516441821/0001193125-16-441821.txt'],
 '0001193125-15-350718': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312515350718/0001193125-15-350718.txt'],
 '0001193125-15-272806': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312515272806/0001193125-15-272806.txt'],
 '0001193125-15-144151': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312515144151/0001193125-15-144151.txt'],
 '0001193125-15-020351': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312515020351/0001193125-15-020351.txt'],
 '0001193125-14-380252': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312514380252/0001193125-14-380252.txt'],
 '0001193125-14-289961': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312514289961/0001193125-14-289961.txt'],
 '0001193125-14-157088': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312514157088/0001193125-14-157088.txt'],
 '0001193125-14-018634': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312514018634/0001193125-14-018634.txt'],
 '0001193125-13-409855': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312513409855/0001193125-13-409855.txt'],
 '0001193125-13-310206': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312513310206/0001193125-13-310206.txt'],
 '0001193125-13-160748': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312513160748/0001193125-13-160748.txt'],
 '0001193125-13-022207': ['https://www.sec.gov/Archives/edgar/data/0000789019/000119312513022207/0001193125-13-022207.txt']
}

In [ ]:
# Show AN_Active
AN_Active

In [ ]:
# Before 2017 AND After 0001193125-12-170665
for key, url_AN in AN_Active.items():
            
    AN = url_AN[1]
    url = url_AN[0]
    
    print('key - ', key)
    print('\n')
    print('Accession Number - ', AN)
    print('\n')
    print('Active URL - ', url)
    print('\n')
    
    r = requests.get(url)
    soup = BeautifulSoup(r.content, "lxml")

    filing_document = soup.find('document')
    page_key = soup.find_all(attrs={"name": "{}".format(key.replace('#', ''))})
    
    document_id = filing_document.type.find(text=True, recursive=False).strip()
        
    T_table = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]    
    # print('T_table - ', T_table)
    
    target = T_table[0].findAll('tr')
    row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']

    col_list = []

    for row in row_list:
        col = row.find_all('td')
        column = []
        for tr in col:
            ele = tr.text
            ele= ele.encode('latin1').decode("utf-8", "ignore")
            column.append(ele)
        col_list.append(column)

    col_list.pop(0)

    new_col_list = []
    for col_num, column in enumerate(col_list): 

        if col_num == 0:
            print(column)
            Q, Y = Quarter(column)
            print(Q, Y)
            new_col_list.append(Q)
            new_col_list.append(Y)

        else:
            new_col_list.append(cleaning_column(column))

    df = pd.DataFrame(new_col_list)
    
    df = df.T.reset_index(drop=True)
    df.columns = df.iloc[0]
    df = df.drop([0,2], axis=0).reset_index(drop=True)
    ICD.display(df)

    try: 
        with open('{}-{}.csv'.format(AN, document_id), "wb") as fp:   # Pickling
            pickle.dump(df, fp)

        # display a status to the user.
        print('All the pages from {} document {} have been tableized and saved.'.format(AN, document_id))

    except:
        value_list.append(vv)
        print("Can't save file {}, file type = {}".format(AN, document_id))

        print('-'*80) 


# New Method try 2016 and before, reference material ->

In [ ]:
# Got the key 
test_url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516662209/d187868d10k.htm#tx187868_27'

r = requests.get(test_url)
soup = BeautifulSoup(r.content, "html")

hr_tag = soup.findAll('a', href=True)
list_of_table_in_page = [table.get('href') for table in hr_tag if 'Balance Sheets' in table.get_text() ][0]
print(list_of_table_in_page)

page_key = soup.find_all(attrs={"name": "tx187868_27"})
# add this to table below -> .findAll('tr')
T_table = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
print(T_table)

# Figuring out how to use the key.

In [ ]:
def has_key(name):
    if  re.compile("#tx245252_5").search(name):
        return name 

In [ ]:
test_url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000119312516742796/0001193125-16-742796.txt'

r = requests.get(test_url)
soup = BeautifulSoup(r.content, "lxml")

page_key = soup.find_all(attrs={"name": "tx245252_5"})

T_table = [ table.parent.parent.parent.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]

In [ ]:
page_key

In [ ]:
target = T_table[0].findAll('tr')
print(target)

In [ ]:
type(target)

In [ ]:
row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']
print(row_list)

In [ ]:
col_list = []
for row in row_list:
    col = row.find_all('td')
    column = []
    for tr in col:
        ele = tr.get_text()
        ele= ele.encode('latin1').decode('cp1252')
        column.append(ele)
    col_list.append(column)
    
print(col_list)

In [ ]:
col_list.pop(0)

In [ ]:
new_col_list = []
for col_num, column in enumerate(col_list): 
    
    if col_num == 0:
        print(column)
        Q, Y = Quarter(column)
        print(Q, Y)
        new_col_list.append(Q)
        new_col_list.append(Y)
        
    else:
        new_col_list.append(cleaning_column(column))

In [ ]:
new_col_list

In [ ]:
df = pd.DataFrame(new_col_list)
df = df.T.reset_index(drop=True)
df

In [ ]:
df.columns = df.iloc[0]
df = df.drop([0,2], axis=0).reset_index(drop=True)


## End of old reference material 

# < TESTING AREA>

In [ ]:
# url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/msft-10q_20200331.htm#BALANCE_SHEETS'
url = 'https://www.sec.gov/Archives/edgar/data/0000789019/000156459020019706/0001564590-20-019706.txt'
key = '0001564590-20-019706'

In [ ]:
r = requests.get(url)
soup = BeautifulSoup(r.content, "lxml")

filing_document = soup.find('document')
document_id = filing_document.type.find(text=True, recursive=False).strip()

if key in list(Accession_Number_URL_16_20.keys())[0:3]:
    page_key = soup.find_all(attrs={"id": "BALANCE_SHEETS"})

    T_table = [table.next_sibling.next_sibling.next_sibling.next_sibling for table in page_key ]
    # print(T_table)

else:
    page_key = soup.find_all(attrs={"name": "BALANCE_SHEETS"})

    T_table = [ page_key[0].parent.next_sibling.next_sibling.next_sibling.next_sibling ]

target = T_table[0].findAll('tr')
row_list = [ row for row in target if row.get_text(strip=True, separator=',') != '']

col_list = []

for row in row_list:
    col = row.find_all('td')
    column = []
    for tr in col:
        ele = tr.text
        ele= ele.encode('cp1252').decode("utf-8", "ignore")
        column.append(ele)
    col_list.append(column)

col_list.pop(0)

new_col_list = []
for col_num, r_column in enumerate(col_list): 

    if col_num == 0:
        print('raw column - ', r_column)
        Q, Y = Quarter(r_column)
        print(Q, Y)
        new_col_list.append(Q)
        new_col_list.append(Y)

    else:
        
        new_col_list.append(cleaning_column( r_column ))

df = pd.DataFrame(new_col_list)
df = df.T.reset_index(drop=True)
df.columns = df.iloc[0]
df = df.drop([row for row in range(df.shape[0]) if row != 1], axis=0).reset_index(drop=True)
ICD.display(df)


In [ ]:
new_dict = [name for name in df.columns]
print(new_dict, len(new_dict))

In [ ]:
old_if = [name for name in df.columns]
print(old_if, len(old_if))

In [ ]:
import collections
print([item for item, count in collections.Counter(old_if).items() if count > 1])


In [ ]:
print(set(old_if) - set(new_dict), '\n')
print(set(new_dict) - set(old_if))

# </END TESTING AREA>

#                     {Nexted Function Territory}
## Take care of the multiplier in cleaning_column() and else where
## Limit: slow, not lean, ineffcient, relied heavily on indices (position in a list).

In [ ]:
QorK = {'10-Q' : [],
        '10-K' : []
       }

for acc_num, url in Accession_Number_URL.items():
    
    master_document_dict = {}
    
    # create a stop point
    if acc_num == '0001193125-16-742796':
        break

    else:
        
        # grab the response
        response = requests.get(url[0])

    # Soupify
        # pass it through the parser, in this case let's just use lxml because the tags seem to follow xml.
        soup = BeautifulSoup(response.content, 'lxml')
        filing_document = soup.find('document')
        
    # Parsing
        # Document type ->> document_id
        document_id = filing_document.type.find(text=True, recursive=False).strip()
        # Document sequence ->> document_sequence
        document_sequence = filing_document.sequence.find(text=True, recursive=False).strip()
        # Document filename ->> document_filename
        document_filename = filing_document.filename.find(text=True, recursive=False).strip()
        # Document description ->> document_description
        document_description = filing_document.description.find(text=True, recursive=False).strip()
        
    # Storage    
        # initalize our document dictionary
        master_document_dict[document_id] = {}

        # add the different parts, we parsed up above.
        master_document_dict[document_id]['document_sequence'] = document_sequence
        master_document_dict[document_id]['document_filename'] = document_filename
        master_document_dict[document_id]['document_description'] = document_description
        
# Scraping
        # grab the text portion of the document, this will be used to split the document into pages.
        filing_doc_text = filing_document.find('text').extract()

        # find all the thematic breaks, these help define page numbers and page breaks.
        all_thematic_breaks = filing_doc_text.find_all('table')

        # Locate and store page number via list comprehension.
        all_page_numbers = [thematic_break.previous_sibling for thematic_break in all_thematic_breaks if len(str(thematic_break))>6]
            
        # convert all thematic breaks to a string so it can be used for parsing
        all_thematic_breaks = [str(thematic_break) for thematic_break in all_thematic_breaks]

        # prep the document text for splitting, this means converting it to a string.
        filing_doc_string = str(filing_doc_text)
    
        # handle the case where there are thematic breaks.
        if len(all_thematic_breaks) > 0: 

            # define the regex delimiter pattern, this would just be all of our thematic breaks.
            regex_delimiter_pattern = '|'.join(map(re.escape, all_thematic_breaks))

            # split the document along each thematic break.
            split_filing_string = re.split(regex_delimiter_pattern, filing_doc_string)

            # store the document itself
            master_document_dict[document_id]['pages_code'] = split_filing_string

        # handle the case where there are no thematic breaks.
        elif len(all_thematic_breaks) == 0:

            # handles so it will display correctly.
            split_filing_string = all_thematic_breaks

            # store the document as is, since there are no thematic breaks. In other words, no splitting.
            master_document_dict[document_id]['pages_code'] = [filing_doc_string]
            
        # display some information to the user.
        print('-'*80)
        print('The document {} was parsed.'.format(document_id))
        print('There was {} page(s) found.'.format(len(all_page_numbers)))
        print('There was {} thematic breaks(s) found.'.format(len(all_thematic_breaks)))

        # store the documents in the master_filing_dictionary.
        master_filings_dict[acc_num]['filing_documents'] = master_document_dict
        
        # if document is 10-Q
        if document_id == '10-Q':
            QorK['10-Q'].append(acc_num) # add acc_num to QorK in 10-Q as key
        
        # if document is 10-K
        if document_id == '10-K':
            QorK['10-K'].append(acc_num) # add acc_num to QorK in 10-K as key
        
        del master_document_dict
        
        print('-'*80)
        print('All the documents for filing {} were parsed and stored.'.format(acc_num))
    


In [ ]:
string_test = master_filings_dict['0001193125-16-742796']['filing_documents']['10-Q']['pages_code']
for page_num, page in enumerate(string_test):
    if ('Cash and cash equivalents' in page) and ('Short-term investments' in page):
        print( string_test.index(page_num  + 3) )
        print('~'*40)
#string_test[56]
# df01 = table(string_test, '10-Q')
# df = df01[0]
# df

## Anchor Check Function



In [ ]:
# Variations of HTML anchors.
CBS_A = 'BALANCE SHEETS'
CBS_B = 'name="BALANCE_SHEETS"'
CBS_D = 'BALANCE SHEETS '
CBS_C = 'Cash and cash equivalents'
# SoO_A = '<a name="Statements_of_Operations"' 
# SoO_B = 'id="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# SoO3 = 'name="CONSOLIDATED_STATEMENTS_OPERATIONS"'
# CSoO10K = 'name="Consolidated_Statements_of_Operations"'
# CSoO10Knew = 'id="Consolidated_Statements_of_Operations"'

# CF_A = '<a name="Statements_of_Cash'
# CF_B = 'id="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CF3 = 'name="CONSOLIDATED_STATEMENTS_CASH_FLOWS"'
# CCF10K = 'name="Consolidated_Statements_of_Cash_Flows"'
# CCF10Knew = 'id="Consolidated_Statements_of_Cash_Flows"'

def anchor_check(page_code):
    
    # if the page has one of these anchors
    if (CBS_A in page_code) or (CBS_B in page_code) or (CBS_D in page_code) and (CBS_C in page_code):
        return True

    #elif (SoO_A in page_code) or (SoO_B in page_code) or (SoO3 in page_code) or (CSoO10K in page_code) or (CSoO10Knew in page_code) :
        #return True

    #elif (CF_A in page_code) or (CF_B in page_code) or (CF3 in page_code) or (CCF10K in page_code) or (CCF10Knew in page_code):
        #return True


    else:
        return False


## Cleaning Columns and Cleaning Tables

In [ ]:
def table_extractor( page, multiplier, k ):
    
    # Convert string to BS object
    soup = BeautifulSoup(page, 'html5')

    # then get all the rows in the table.
    table_rows = soup.find_all('tr')
    
    if len( [tr.text for tr in table_rows[0].find_all('td')] ) < 3:
        del table_rows[0]
    
    single_table = []
    Date_column = []
    
    # Loop through each column, adding in three structures to single_table.
    for tr_post, tr in enumerate(table_rows):
        
        td = tr.find_all('td')

        column = [tr.text for tr in td]
        #print( 'tr_post', tr_post, len(column), column)
        # 10-Q 
        if k == '10-Q':
            
            if tr_post == 3:

                # Quarter Column
                col_Q, col_year = Quarter( column )
                single_table.append( col_Q )
                # Year Column
                single_table.append( col_year )

            elif tr_post > 4: 

                # Features Columns
                cleaned_data = cleaning_column( column , multiplier )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                    
                single_table.append( cleaned_data )
                
        # 10-K
        elif k == '10-K':
            
#             if tr_post == 1 or tr_post == 2:
                    
#                 Date_column.append(column)
                
            if tr_post == 3:
                
                # Quarter Column
                col_date, col_year = Quarter(column)
                single_table.append( col_date )
                # Year Column
                single_table.append( col_year )
                
            elif tr_post > 4:              
                
                cleaned_data = cleaning_column( column , 10**6 )
                if len(cleaned_data) > 10:
                    del cleaned_data[3]
                
                single_table.append( cleaned_data )
            
    return single_table

In [ ]:
def df_clean( single_table ):
    print(single_table)
    table_df = pd.DataFrame(single_table)

    table_df = table_df.transpose()  
    print(table_df)
    table_df.drop(table_df.index[1:3], 0, inplace=True)

    #table_df.drop(table_df.index[2:], 0, inplace=True)

    table_df.columns = table_df.iloc[0]

    table_df = table_df.drop(table_df.index[0])
    
    return table_df

In [ ]:
def table(page_code, k):
    
    doc_tables=[]

    for page_post, page in enumerate(page_code):
        
        if ('(In millions) (Unaudited)' in page) and ('BALANCE SHEETS ' in page):
            
            single_table = table_extractor( page, 10**6, k )
            #print(single_table)
            table_df = df_clean(single_table)
                        
            doc_tables.append(table_df)
        
        elif ('BALANCE SHEETS ' in page) and ('(In thousands) (Unaudited)' in page):
            
            #print('page_post', page_post)
            
            single_table = table_extractor( page, 10**3, k )

            table_df = df_clean(single_table)

            doc_tables.append(table_df)
            
        else:
            print(page_post+1, 'anchor_check / in thousands/million problem')

                
    return doc_tables
    

# Exploring the Data

In [ ]:
explore_dict = {}
for QK, AccNum_list in QorK.items():
    
    #for an_postition, acc_num in enumerate(AccNum_list): 
    if QK== '10-Q':
        Q_new = AccNum_list[0]
        Q_old = AccNum_list[-1]
        explore_dict[QK] = [Q_new, Q_old]

    if QK == '10-K':
        K_new = AccNum_list[0]
        K_old = AccNum_list[-1]
        explore_dict[QK] = [K_new, K_old]

In [ ]:
QorK ={'10-Q': ['0001564590-20-019706',
  '0001564590-20-002450',
  '0001564590-19-037549',
  '0001564590-19-012709',
  '0001564590-19-001392',
  '0001564590-18-024893',
  '0001564590-18-009307',
  '0001564590-18-001129',
  '0001564590-17-020171',
  '0001564590-17-007547',
  '0001564590-17-000654'],
 '10-K': ['0001564590-19-027952',
  '0001564590-18-019062',
  '0001564590-17-014900']}

In [ ]:
explore_dict

In [ ]:
for QK, N_O in explore_dict.items():
    
    for an_postition, acc_num in enumerate(N_O):
        
        pages = master_filings_dict[acc_num]['filing_documents'][QK]['pages_code']
        print('\n', 'This is {} a {} document.'.format(acc_num, QK))
        test_df_list = table(pages, QK)
        test_df = test_df_list[0]
        
        table_df = test_df.drop(test_df.index[1:]).reset_index(drop=True)
        print('Shape - ', table_df.shape)
        ICD.display(table_df)
        

In [ ]:
Quarter(['\n\xa0', '\n\xa0', '\nMarch 31,2020', '\n\xa0', '\n\xa0', '\nJune\xa030,2019', '\n\xa0'])

# Save Files Locally. 

In [ ]:
Checklist = {}

for k, v in QorK.items():
    value_list = []
    for i, vv in enumerate(v):

        # first grab 10-Q documents only
        pages = master_filings_dict[v[i]]['filing_documents'][k]['pages_code']

        try:
            # Extract Table from html code and wrap it as a list of df.
            test_df_list = table(pages, k)
            test_df = test_df_list[0]
            table_df = test_df.drop(test_df.index[1:]).reset_index(drop=True)
            #ICD.display(table_df[0])

            with open('{}-{}.csv'.format(v[i], k), "wb") as fp:   # Pickling
                pickle.dump(table_df, fp)

            # display a status to the user.
            print('All the pages from {} document {} have been tableized and saved.'.format(k, v[i]))
        
        except: 
            value_list.append(vv)
            print('Something went wrong in file {}, file type = {}'.format(v[i], k))
        
        print('-'*80) 
    Checklist['{}'.format(k)] = value_list
    del value_list

In [ ]:
# Checklist of erroneous filings.  
for a, b in Checklist.items():
    if b != []:
        print('These have a problem.', '\n', a, b)

    else: 
        print(a, 'has no problem!! :) ' , '\n')
    
print(Checklist)

In [ ]:
# Bring QorK over to Part 2. 
QorK